In [ ]:
# Import the required libraries
import bs4
import urllib3
import csv
import numpy as np
urllib3.disable_warnings()
http = urllib3.PoolManager()

# ntlk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist

#### Sanitizing and Sorting Output

In [ ]:
def sanitize_input(data):
    replace = {
        ord('\f') : ' ', 
        ord('\t') : ' ',
        ord('\n') : ' ',
        ord('\r') : None
    }
    return data.translate(replace)

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def tokenize_content(content):
    stop_words = set(stopwords.words('english') + list(punctuation))
    words = word_tokenize(content.lower())
    return [word for word in words if word not in stop_words]

def word_freq(content):
    filtered_words = tokenize_content(content)
    word_freq = FreqDist(filtered_words)
    return word_freq

def filterWords(allWords):
    removewords = ['p','div','while','total','been','e','our','like','new','which','help','s','all','some','if','what','about','only','on','then','will','no','at','a','for','us','not','etc','we','that','it','the','of','as','an','may','have','has','this','other','from','with','its','be','in','is','am','now','you','some','was','can','are','but','they','he','she','where','when','and','or','them','how','by','to']
    selWords = [word for word in allWords if word not in removewords]
    selWords = [word for word in selWords if not str.isdigit(word)]
    return selWords
def filtercharacter(string):
    removechar = ['"',"'",'?',',','‘','’','-','(',')',':','—','/','<','>']
    string = [c if c not in removechar else ' ' for c in string ]
    return ''.join(string)

#### Link of Pages with list of JDs

In [ ]:
def url_query(job):
    Url_List = []
    job = job.strip().replace(" ","+")
    base_url = 'https://www.indeed.co.in/jobs?l=India&q='
    job_url_default = base_url+job
    Url_List.append(job_url_default)
    job_url_page = job_url_default+'&start='
    for i in range(1,2):             
        Url_List.append(job_url_page+str(10*i))
    return Url_List

#### Extract JD from given List

In [ ]:
def JDfrompage (page_url):
    r = http.request('GET', page_url)
    source = r.data
    soup = bs4.BeautifulSoup(source, "lxml")
    all_div = [div for div in soup.findAll('div',{"class": "jobsearch-SerpJobCard"}) ]
    all_jk = ['https://www.indeed.co.in/viewjob?jk='+div.get('data-jk') for div in all_div]
    return all_jk

#### All JD of given job

In [ ]:
def get_JDs(JD_URL_List):    
    res = []
    for url in JD_URL_List:
        print(url)
        r = http.request('GET',url)
        source = r.data
        if(r.status==200):
            soup = bs4.BeautifulSoup(source, "lxml")
            JD = soup.find('div',{"class": "jobsearch-JobComponent-description"})
            JD_content = JD.contents[0].encode('utf-8').decode()
            JD_content = sanitize_input(JD_content)
            JD_content = remove_html_tags(JD_content)
            res.append(JD_content)
    return res

#### Creating list of all jobs and Skills

In [ ]:
Jobtype = ["Android Developer"
           ,"Web Developer"
           ,"Consultant"
           ,"Finance"
           ,"Data Scientist"
           ,"Business Analyst"
           ,"Designer"
           ,"Mechanical Engineer"
           ,"Electrical Engineer"
           ,"Civil Engineer"
           ,"Chemical Engineer"
           ,"Software Developer"
           ,"Game Developer"]
Skills = ['Mumbai'
         ,'Delhi'
         ,'Bangalore'
         ,'Kochi'
         ,'Noida']

In [ ]:
JobAndJDs = {}
for job in Jobtype:
    JD_URL_List = []
    for url in url_query(job):
        x = JDfrompage(url)   # list of all jobs from that url
        JD_URL_List = JD_URL_List+x
    all_JD = get_JDs(JD_URL_List)
    JobAndJDs[job] = all_JD
    print(job+ ":- Total JDs = "+str(len(all_JD)))

In [ ]:
def skillinJd(jd,skills):
    skillMatrix = []
    for c in skills:
        if(jd.count(c)>0):
            skillMatrix.append(1.0)
        else:
            skillMatrix.append(0.0)
#     print(np.array(skillMatrix))
    return np.array(skillMatrix)

In [ ]:
CareerSkillMatrix = {}
for career in JobAndJDs.keys():
    SkillMatrix = np.array([0]*len(Skills))
    for jd in JobAndJDs[career]:
        SkillMatrix = np.add(SkillMatrix,skillinJd(jd,Skills))
    SkillMatrix = np.divide(SkillMatrix,len(JobAndJDs[c]))
    CareerSkillMatrix[career] = SkillMatrix